In [29]:
import pandas as pd
import os
import time
import numpy as np
import librosa
import librosa.display
from tensorflow.keras.utils import to_categorical

In [54]:
data_path = r"E:\personal\LiveProjects\speech_recognition\google_speech_new\google_speech\test"
csv_name = r"test.csv"
#data_path_train = r"E:\personal\LiveProjects\speech_recognition\google_speech_new\google_speech\train"
#csv_name_train = r"train.csv"

train_csv = os.path.join(data_path, csv_name)
df = pd.read_csv(train_csv, encoding='utf-8')
#print(df)
# Function to return Mel spectogram of given audio file
def get_mel_spectogram(data, samplerate):
    return librosa.feature.melspectrogram(y=data, sr=samplerate)

def get_mel_spectogram_from_audio_path(audio_path):
    data, samplerate = librosa.load(audio_path)
    M = get_mel_spectogram(data, samplerate)
    return M

# Generate mel spectogram for all training audio files
def make_1s_duration(data, duration):
    return librosa.effects.time_stretch(data, duration)

startTime = time.time()
x_train = np.array([])
y_train = pd.get_dummies(df['label'])
y_train = y_train.to_numpy()

#for ind in df.index:
#for ind in np.arange(100):
def featurize_clip1(fp):
    #fp = df.loc[ind, "file_path"]
    audio_path = os.path.join(data_path, os.path.join(r"audio", fp))
    #print(audio_path)
    data, samplerate = librosa.load(audio_path, sr=None)
    #print(samplerate)
    duration = librosa.get_duration(y=data, sr=samplerate)
    #print(duration)
    #print(data.shape[0])
    data_stretch = make_1s_duration(data, duration)
    #print(data_stretch.shape)
    M = get_mel_spectogram(data_stretch, samplerate)
    #print(np.shape(M))
    #if ind == 0:
    #    x_train = M
    #else:
    #    x_train = np.vstack((x_train, M)).astype(np.float32)
    #y_train[ind] = df.loc[ind, "label"]
    return M

train_audio_path = f"{data_path}/audio"

def pad_audio_with_silence(audio: np.ndarray, sample_rate: int, duration: float):
    target_n_samples = int(duration * sample_rate)
    #assert audio.shape[0] < target_n_samples
    padded_audio = np.zeros(target_n_samples)
    padded_audio[:audio.shape[0]] = audio
    return padded_audio

def mel_spectrogram(
    audio: np.ndarray,
    sample_rate: int,
    threshold: int=None,
):
    melspec = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
    melspec = librosa.power_to_db(melspec, ref=np.max)
    if threshold:
        melspec[melspec < threshold] = -80
    return melspec

def featurize_clip(file_path):
    audio, sample_rate = librosa.load(f"{train_audio_path}/{file_path}", sr=None)
    audio = pad_audio_with_silence(audio, sample_rate, duration=1)
    spectrogram = mel_spectrogram(audio, sample_rate)
    return spectrogram

df["spectrogram"] = df["file_path"].apply(featurize_clip)

print(np.shape(df["spectrogram"]))
#x_train = x_train.reshape((-1, 64, 64, 1))


print(y_train)
#y_train = to_categorical(y_train)
#print(np.shape(y_train))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

(6835,)
[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]
Execution time in seconds: 38.64708995819092


In [55]:
from tensorflow import keras 
from tensorflow.keras import layers
#from tensorflow.keras.layers import Rescaling
#from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [70]:
n_classes = df["label"].unique().shape[0]
n_classes

inputs = keras.Input(shape=(128, 32, 1))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
#x = layers.MaxPooling2D(pool_size=2)(x)
#x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
#x = layers.MaxPooling2D(pool_size=2)(x)
#x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
#x = layers.MaxPooling2D(pool_size=2)(x)
#x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(units=n_classes, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 128, 32, 1)]      0         
                                                                 
 rescaling_7 (Rescaling)     (None, 128, 32, 1)        0         
                                                                 
 conv2d_25 (Conv2D)          (None, 126, 30, 32)       320       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 63, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 61, 13, 64)        18496     
                                                                 
 flatten_6 (Flatten)         (None, 50752)             0         
                                                           

In [71]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [72]:
# network parameters
#input_shape = (64, 64, 1)
batch_size = 128
kernel_size = 3
filters = 64
dropout = 0.3

spectrogram_shape = df["spectrogram"].iloc[1].shape
spectrogram_shape

input_shape = (spectrogram_shape[0], spectrogram_shape[1], 1)
input_shape

X = np.stack(df["spectrogram"])
X.shape

(6835, 128, 32)

In [73]:
#x_train = df["spectrogram"].to_numpy()

#print(x_train[0].shape)

#def stack_it(arr):
#    return arr.shape()
#df["spectrogram_processed"] = df["spectrogram"].apply(stack_it)
import tensorflow as tf
class DataframeSequence(keras.utils.Sequence):
    def __init__(self, df, y_train, batch_size):
        self.df = df
        self.batch_size = batch_size
        self.y_train = y_train

    def __len__(self):
        return int(np.ceil(len(self.df) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.df['spectrogram'][idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y_train[idx * self.batch_size:(idx + 1) * self.batch_size]
        print(batch_x.shape)
        return tf.convert_to_tensor(batch_x), tf.convert_to_tensor(batch_y)

sequence = DataframeSequence(df, y_train, batch_size)


In [74]:
# train the model with input images and labels
print(x_train.shape)
#model.fit_generator(sequence, epochs=20)
model.fit(X,
          y_train,
          #validation_data=(x_test, y_test),
          #sequence,
          epochs=20,
          batch_size=batch_size)


(0,)
Epoch 1/20
54/54 [==============================] - 32s 585ms/step - loss: 3.2523 - accuracy: 0.0969
Epoch 2/20
54/54 [==============================] - 35s 651ms/step - loss: 2.3202 - accuracy: 0.3606
Epoch 3/20
54/54 [==============================] - 34s 635ms/step - loss: 1.8607 - accuracy: 0.4808
Epoch 4/20
54/54 [==============================] - 40s 746ms/step - loss: 1.6531 - accuracy: 0.5388
Epoch 5/20
54/54 [==============================] - 41s 758ms/step - loss: 1.4854 - accuracy: 0.5899
Epoch 6/20
54/54 [==============================] - 44s 812ms/step - loss: 1.3541 - accuracy: 0.6209
Epoch 7/20
54/54 [==============================] - 42s 779ms/step - loss: 1.2247 - accuracy: 0.6631
Epoch 8/20
54/54 [==============================] - 42s 783ms/step - loss: 1.0900 - accuracy: 0.6938
Epoch 9/20
54/54 [==============================] - 43s 804ms/step - loss: 0.9692 - accuracy: 0.7276
Epoch 10/20
54/54 [==============================] - 42s 772ms/step - loss: 0.8411 - a